# Project 1, Part 5, Data Visualization



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [26]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [27]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [28]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.5 Example of a Data Visualization created using Python

The data science team would like for you to create an example of a data visualization using Python from data in a Pandas dataframe containing data from an SQL query.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  Any query of your choosing.  You can write a query from scratch.  You can use a query from a previous problem in this project.  You can use a query from the labs.  The idea is to come up with a query whose resulting data will make for an excellent quality data visualization. 

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.  Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.

Once you have the data in a Pandas dataframe, you may write as much Python code and use as many code cells as you wish to produce the data visualization.

You may only use Python modules that are currently installed in the Anaconda Docker container.  You may not install additional modules or any other software.

All work must be done in Docker in your VM in AWS.  You may not use any external data visualization systems, such as Tableau, etc.

You may use any code from the labs to pattern your code after, however for the data visualization, you cannot wholesale copy a data visualization from the labs.

Ensure that it is properly titled, including titles for axes if present.

Ensure that when you check this Juptyer Notebook into GitHub that the data visualization is clearly visible.  

If you want to use Google Maps for your data visualization, this is fine, however, the image will not show up in GitHub.  So, just save the image to an image file, include it in the repo, and add a markdown cell to display the image file. Also with Google Maps, do NOT check gmap_api_key.txt into GitHub for security reasons.  If the grader needs to run it, they will supply their own gmap_api_key.txt.


In [29]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with helper1 as (
                  select zip, 
                         latitude, 
                         longitude,
                         city,
                         state,
                         population as zip_population
                  from zip_codes
                  group by zip
                ),
    helper2 as  (
                  select zip,
                         count(*) as customer_population
                  from customers
                  group by zip
                ),
    helper3 as (
                 select a.zip,
                        a.customer_population,
                        b.zip_population,
                        b.latitude,
                        b.longitude,
                        b.city,
                        b.state,
                        customer_population * 100.0 / zip_population as percentage_customers_per_population_raw
                 from helper2 as a
                 join helper1 as b
                 on a.zip = b.zip
               )
select zip,
       city,
       state,
       latitude,
       longitude,
       round(percentage_customers_per_population_raw, 3) as percentage_customers_per_population
from helper3
order by percentage_customers_per_population_raw desc

"""


df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,zip,city,state,latitude,longitude,percentage_customers_per_population
0,98164,Seattle,WA,47.6060,-122.3320,1.290
1,98050,Preston,WA,47.5440,-121.9404,1.087
2,33109,Miami Beach,FL,25.7614,-80.1425,1.053
3,94613,Oakland,CA,37.7809,-122.1829,1.045
4,37240,Nashville,TN,36.1449,-86.8055,1.028
...,...,...,...,...,...,...
545,33033,Homestead,FL,25.4837,-80.4136,0.002
546,75067,Lewisville,TX,33.0137,-97.0002,0.001
547,75035,Frisco,TX,33.1553,-96.7727,0.001
548,94565,Pittsburg,CA,38.0158,-121.9070,0.001


In [30]:
# Use as many code cells as you need to create and display your data visualization


In [31]:
import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

In [32]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [33]:
states = df['state'].unique()
figs = []

for index in range(0, len(states)):
    df_state = df[df['state'] == states[index]]
    center = (df_state['latitude'].mean(), df_state['longitude'].mean())
    fig = gmaps.figure(center=center, zoom_level=10)
    
    locations = df_state[['latitude','longitude']]
    weights = df_state[['percentage_customers_per_population']].to_numpy().reshape(len(df_state))
    heatmap_layer = gmaps.heatmap_layer(locations=locations, weights=weights,point_radius=20)
    fig.add_layer(heatmap_layer)
    figs.append(fig)

In [34]:
pwd

'/user/projects/project-1-vibhatna'

In [39]:
import os
files = os.listdir()
files

['.git',
 'README.md',
 'project_1_1.ipynb',
 'project_1_2.ipynb',
 'project_1_3.ipynb',
 'project_1_4.ipynb',
 'project_1_5.ipynb',
 'project_1_6.ipynb',
 '.ipynb_checkpoints',
 'gmap_api_key.txt',
 'WA.png',
 'TX.png',
 'CA.png',
 'FL.png',
 'TN.png']

<img src=https://github.com/mids-w205/project-1-vibhatna/blob/project/WA.png \>